### Imports

In [1]:
#began 5/1/2020 k.ames and used by J.Over. Needs fix for unequal point spacing and multiple time steps

import os
import csv
import joblib
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pybeach.beach import Profile
from pybeach.support import data_support as ds
from pybeach.support import classifier_support as cs
%matplotlib inline


### Load Data
The code will need one csv with the following fields named:<br>
__'X'__ - easting of each point<br>
__'Y'__ - northing of each point<br>
__'TransOrder'__ - transect number of each point<br>
__'Elevation'__ - elevation of each point<br>
__'Info'__ - year of each point in yyyymmdd format 

Previously have run into issues with by beach throwing an warning that it looked like the data were organized seaward to landward. Pybeach will only work with landward to seaward data. If this warning persisits after double checking data are in the correct direction try clipping the data nearer the landward side of the dune crest. The error "All NaN slice" occurs if data is organized seaward to landward or if there is no dune crest on the transect.

In [47]:
#-------------------USER EDITS---------------------------------
year = 20190830 # User entered year, matching values in the info field
csvname = "NCB_Profiles_test_7.csv" #csvname
total_dist = 200 #length of longest transect (m)
spacing = 1 #spacing of points e.g. 1m, 2m
export = "y" #(y/n) export figures created
path = 'C:/Users/jover/programs/Pybeach_Working/Notebook_pybeach_figures' #path where csv will export
#---------------------------------------------------------------

#This is setting up a distance array. FIIS elevations were pulled every 2m so this works
x = np.arange(0, total_dist, spacing) 

NCB_array = pd.read_csv(csvname,  sep=',')
Data_sel=NCB_array[NCB_array["Info"]==year]         #split out one year of data
Transects = sorted(list(set(Data_sel["TransOrder"]))) #Keep unique transect values for looping

z_list=[]
east_list=[]
north_list=[]

for n in Transects:
    trans_sel=Data_sel[Data_sel["TransOrder"]==n]   #isolate data from one transect    
    z_ind=trans_sel["Elevation"] #get all values associated with that transect and year
    east_ind=trans_sel["X"]
    north_ind=trans_sel["Y"]
    
    z1 =z_ind.values.tolist() #convert to list so we can put it back into a different array
    east1 =east_ind.values.tolist()
    north1 =north_ind.values.tolist()
    
    if len(z1) < (total_dist/spacing):  #padding with NaNs, not all transects are the same length
        #addon=np.repeat(np.nan, (total_dist/spacing)-len(z1))    
        addon=np.repeat(-0.03, (total_dist/spacing)-len(z1))                                       
        z1.extend(addon)
        east1.extend(addon)
        north1.extend(addon)
    
    z_list.append(z1) #append onto lists
    east_list.append(east1)
    north_list.append(north1)

z=np.array(z_list) #put back into arrays for pybeach to use
east=np.array(east_list)
north=np.array(north_list)
print("Created np arrays of Easting, Northing and Elevation")
#print (z) #print commands for error checking
#np.savetxt("foo.csv", z, delimiter=",")
print (z_ind)
#print (north)

Created np arrays of Easting, Northing and Elevation
60353    2.336
60356    2.330
60359    2.407
60362    2.510
60365    2.510
         ...  
60560    2.153
60561    1.976
60562    1.976
60563    1.751
60564    0.400
Name: Elevation, Length: 132, dtype: float64


### Predicting dune toes 

In [44]:
# Instantiate Profile class
p = Profile(x, z)
# Make predictions of dune toe

toe_ml, prob_ml = p.predict_dunetoe_ml('barrier_island_clf', dune_crest='max', window_size=50, threshold=0.97)
toe_mc = p.predict_dunetoe_mc(dune_crest='max', window_size=100, threshold=0.98,)
toe_rr = p.predict_dunetoe_rr(dune_crest='max', window_size=[23,24,100], threshold=0.98, toe_threshold=0.2, verbose=False)
toe_pd = p.predict_dunetoe_pd(dune_crest='max', window_size=23, threshold=0.98)
# Predict dune crest
crest = p.predict_dunecrest()
# Predict shoreline
shoreline = p.predict_shoreline(dune_crest='rr')

### Example plots and results 

In [45]:
# Plotting Paramaters
params = {'legend.fontsize': 12,
           'axes.labelsize': 14,
           'axes.labelweight': 'bold',
           'axes.titlesize': 14,
           'axes.titleweight': 'bold',
           'xtick.labelsize': 14,
           'ytick.labelsize': 14,
           'mathtext.default':'regular'}
plt.rcParams.update(params)

In [48]:
# Specify profile to plot
n = 0
for n in range(0,len(z)): #number of transects, iterate over
    figurename="Transect_" + str(Transects[n]) +"_" + str(year) + ".pdf"
    # Figure metadata
    fig, axes = plt.subplots(1, 1, figsize=(7, 5))
    toes = [toe_ml[n],toe_mc[n], toe_rr[n], toe_pd[n]] #
    labels = ['Machine Learning','Maximum curvature', 'Relative relief', 'Perpendicular distance'] #
    colors = ['k', 'tomato', 'cornflowerblue', 'gold', 'limegreen'] #
    # Generate figure
    axes.plot(x, z[n], '-k')
    axes.fill_between([70, 100], [0, 0], y2=-1, color='lightskyblue', alpha=0.5)
    axes.fill_between(x, z[n], y2=-1, color='cornsilk', alpha=1)
    axes.axvspan(-10, -9, color='tomato', alpha = 0.6, label='ML Toe probability') # legend placeholder
    for i, itoe in enumerate(toes): 
        axes.plot(x[itoe], z[n, itoe],
                  'o', color=colors[i], ms=12, mec='k', label=labels[i])
    axes.plot(x[crest[n]], z[n, crest[n]], 'v', color='k', ms=12, mec='k', label='Crest')
    axes.plot(x[shoreline[n]], z[n, shoreline[n]], '^', color='k', ms=12, mec='k', label='Shoreline')
    axes.set_xlim(0, 200)
    axes.set_ylim(-.03, 7)
    axes.set_title("Transect-" + str(Transects[n]) + " " + str(year))
    axes.set_xlabel('Cross-shore distance (m)')
    axes.set_ylabel('Elevation (m)')
    axes.grid()
    axes.legend(framealpha=1)
    # Plot dune toe ML probability
    axes2 = axes.twinx() 
    axes2.fill_between(x, prob_ml[n], color='tomato', alpha=0.5)
    axes2.set_ylim(0, 6)
    axes2.set_yticks([0, 1])
    axes2.tick_params(axis='y', labelcolor='tomato')
    axes2.yaxis.set_label_coords(1.1, 0.1)
    axes2.set_ylabel('Toe probability', color='tomato', rotation=270);
    if export == "y":
        fig.savefig(figurename, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.close(fig)
print("Exporting Finished")

Exporting Finished


### Export Data

In [49]:
#set up array with all results for one year of data
titles=('MachineLearning','MaximumCurvature','RelativeRelief','PerpendicularDistance','Crest','Shoreline') #variable names to label

#set up array of index values for all variables
all_results = pd.DataFrame(index=Transects)
all_results[titles[0]] = toe_ml
all_results[titles[1]] = toe_mc
all_results[titles[2]] = toe_rr
all_results[titles[3]] = toe_pd
all_results[titles[4]] = crest
all_results[titles[5]] = shoreline
#all_results.head() #for error checking

#set up blank variables
trans_ind=[]
toe_dist=[]
toe_x=[]
toe_y=[]
toe_z=[]
var_type=[]

#loop through both variables and every transect, labeling what type 
for v in range(0,len(titles)): #for number of headers iterate over
    title="PyBeach_Results_" + str(year) + ".csv"

    for n in range(0,len(z)): #number of transects, iterate over
        ind = all_results.iloc[n,v]
        trans_ind.append(Transects[n])
        toe_dist.append(x[ind])
        toe_x.append(east[n,ind])
        toe_y.append(north[n,ind])
        toe_z.append(z[n,ind])
        var_type.append(titles[v])
        
#create data frame and export csv
df=pd.DataFrame(list(zip(trans_ind, toe_dist, toe_x, toe_y, toe_z, var_type)),columns=['trans_ind','Dist','X','Y','Z','Type'])
df.to_csv(path + title)
df.head() #preview dataframe


,trans_ind,Dist,X,Y,Z,Type
0,2,55,380440.1703,3858381.919,1.666,MachineLearning
1,3,24,380458.5919,3858436.183,2.659,MachineLearning
2,4,44,380511.1391,3858452.547,2.604,MachineLearning
3,5,56,380558.3333,3858474.857,1.803,MachineLearning
4,6,57,380598.1671,3858505.341,1.825,MachineLearning
